## 효과적인 명암비 조절 방법
* 밝은 픽셀은 더욱 밝게, 어두운 픽셀은 더욱 어둡게 조절 (명암대비 높이기)
* ex1) 중간값인 **128**을 기준으로 설정
    * for 명암대비 크게 ==> 128보다 크면 더 밝게, 작으면 더 어둡게
    * for 명암대비 크게 ==> 반대로
* ex2) **평균 밝기**를 구하여 기준으로 설정

* **cv2.convertScaleAbs()** 사용  
    * 절대값으로 변환 후, uint8 자료형으로 바꿔줌  
    * **y = cv2.convertScaleAbs(x, a, b)** ==> **a와 b는 각각 기울기와 y절편**

* **dst = saturate(src + (src - 128) * a)**
    * if) **-1 <= a < 0** : 기울기가 0~1 사이인 직선 ==> **명암비 감소**
    * if) **a > 0** : 기울기가 1보다 큰 직선 ==> **명암비 증가**

![img](effective_contrast.png)

빨간부분 : cv2.multiply()를 이용한 기본적인 명암비 조절  
파란부분 : 128을 기준으로 한 효과적인 명암비 조절 (a 값 이용)
> saturate 되는 부분이 많아서 오히려 구분이 어려워졌던 이전 방법과 달리, 효과적인 명암비 조절을 하면 (명암비 감소 시) 어두운 부분은 조금 더 밝게, 밝은 부분은 조금 더 어둡게 조절 가능!

In [1]:
# a값 1.0을 사용하여 레나 영상의 명암비 증가 ==> cv2.convertScaleAbs() 사용

import cv2
import numpy as np

def contrast2():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    alpha = 1.0
    # saturate(src + (src - 128) * a)
    dst = cv2.convertScaleAbs(src, alpha = 1+alpha, beta = -128*alpha)
    
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
contrast2()

# 히스토그램 분석

# 히스토그램 구하기
* 히스토그램(histogram) : 영상의 픽셀 값 분포를 그래프로 표현한 것 
* 빈(bin) : 그래프에서 가로축 (ex. 0~255 : 빈 개수는 일반적으로 256)
* **cv2.calcHist()** 로 히스토그램 구하기
    * 다양한 형식의 히스토그램 생성 지원해서 사용법 복잡
* **여러 장의 영상**을 입력으로 받을 수 있으므로, **영상들 list**로 전달
* **채널**도 다중 채널, **histSize (빈 개수=히스토그램 크기)**도 **list**로 전달 (각 차원별로 리스트에 담아서)

In [2]:
# Grayscale 영상의 히스토그램 (배열) 구하기

def calcGrayHist(img):
    channels = [0] # 채널에 1개밖에 (0번째 채널밖에) 없으므로 [0] 채널만
    histSize = [256] # 빈 개수 (히스토그램 배열 크기) ==> 채널 1개밖에 없으므로 1개만
    histRange = [0, 256] # 히스토그램 범위
    
    hist = cv2.calcHist([img], channels, None, histSize, histRange)
    
    return hist

img = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
calcGrayHist(img) # CV_32FC1 타입의 (256 x 1) 크기의 행렬 return 

array([[0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [0.000e+00],
       [2.000e+00],
       [9.000e+00],
       [1.300e+01],
       [4.800e+01],
       [8.900e+01],
       [1.250e+02],
       [1.740e+02],
       [2.330e+02],
       [3.610e+02],
       [4.810e+02],
       [5.700e+02],
       [7.630e+02],
       [9.990e+02],
       [1.085e+03],
       [1.316e+03],
       [1.590e+03],
       [1.803e+03],
       [1.939e+03],
       [2.119e+03],
       [2.201e+03],
       [2.244e+03],
       [2.289e+03],
       [2.242e+03],
       [2.258e+03],
       [2.138e+03],
       [2.021e+03],
       [1.985e+03],
       [1.829e+03],


In [3]:
# 위에서 구한 히스토그램으로 히스토그램 영상(막대그래프) 구하기

def getGrayHistImage(hist):
    _, histMax, _, _ = cv2.minMaxLoc(hist) # hist 배열에서 최대값만 구하기
    
    imgHist = np.ones((100, 256), np.uint8) * 255 # 흰색으로 초기화된 영상 생성
    
    # imgHist.shape = (100, 256) ==> 256개 하나씩 이미지 기준 0~255 오른쪽으로 ㄱㄱ
    # 256개 각각의 hist 배열값에 대해서, x축값은 같고, y값만 다른 두 점 이음!
    # 즉, 수직선들(얇은 막대 그래프 선들) 그리면서 나아가므로 검은색으로 칠해진것처럼 보임
    
    for x in range(imgHist.shape[1]): 
        pt1 = (x, 100) # (x, y) ==> (w, h) / 맨 바닥 시작점
        pt2 = (x, 100 - int(hist[x, 0] * 100 / histMax)) # 100px보다 짧은 직선
        cv2.line(imgHist, pt1, pt2, 0) # for문과 line 함수로 그래프 그림(두점씩 선으로)
        
    return imgHist 

hist_img = getGrayHistImage(calcGrayHist(img))

cv2.imshow('hist_img', hist_img)
cv2.imwrite('hist_img.png', hist_img)
cv2.waitKey()
cv2.destroyAllWindows()

# if) 20만큼 밝게 처리한 영상의 경우, 히스토그램 그래프 이미지가 오른쪽으로 20만큼 이동

![img](hist_img.png)

## 히스토그램 분석
* 명암비가 **높은** 영상 : 그래프가 0~255에 걸쳐 **골고루** 분포
* 명암비가 **낮은** 영상 : 그래프가 한 범위에만 **뭉쳐서** 분포

## 히스토그램 스트레칭
* 영상의 히스토그램이 그레이스케일 **전 구간에 걸쳐 골고루** 나타나도록 변경하는 **선형 변환** 기법 ==> for **명암비가 높아지도록!** (좋은 이미지)
* **dst = ((src - min) / (max - min)) * 255**
> 양방향으로 늘려서 Gmin -> 0, Gmax -> 255로 변환하는 것  
* **(Gmin, 0)과 (Gmax, 255)를 지나는 직선의 방정식** 구하는 방식으로 위의 식 구할 수 있다.
    > **기울기(alpha) & Y절편(beta)** 구하기 (**cv2.convertScaleAbs()**의 인자)  
* Gmax(최댓값) & Gmin(최솟값) ==> By. **cv2.minMaxLoc()** 으로 구하기  
(OpenCV에서 스트레칭 함수 미제공)

In [6]:
# 히스토그램 스트레칭 구현

def histogram_stretching():
    src = cv2.imread('hawkes.bmp', cv2.IMREAD_GRAYSCALE)
    
    gmin, gmax, _, _ = cv2.minMaxLoc(src)
    
    # 히스토그램 스트레칭
    dst = cv2.convertScaleAbs(src, alpha=255.0/(gmax-gmin), beta=-gmin*255.0/(gmax-gmin))
    
    cv2.imshow('src', src)
    cv2.imshow('srcHist', getGrayHistImage(calcGrayHist(src)))
    cv2.imwrite('srcHist_img.png', getGrayHistImage(calcGrayHist(src)))
    
    # 스트레칭한 히스토그램 구한 후 히스토그램 이미지 구하기
    cv2.imshow('dst', dst)
    cv2.imshow('dstHist', getGrayHistImage(calcGrayHist(dst)))
    cv2.imwrite('dstHist_img.png', getGrayHistImage(calcGrayHist(dst)))
    
    cv2.waitKey()
    cv2.destroyAllWindows()
    
histogram_stretching()

![src](srcHist_img.png)
![src](dstHist_img.png)

## 히스토그램 평활화 (histogram equalization)
* 픽셀 값 분포가 전체 영역에서 골고루 나타나도록 변경해주는 알고리즘 중 하나
* *히스토그램 스트레칭* : 단지 양쪽으로 펴 주는 것. (평활화가 좀 더 섬세한 알고리즘)
* **히스토그램 h(g)** 로부터 **히스토그램 누적함수 H(g)**를 구해야 함! (시그마)
    * cf. h(g)는 밝기값 g (0~255)에서의 픽셀 개수
* 누적함수 H(g)의 최댓값이 **255**가 되도록 **정규화** (0~255 범위보다 함수값 커지므로)
> **dst = round(H(src) / N * Lmax)**  
* N : 전체 픽셀 개수, Lmax : 영상이 가질 수 있는 최대 밝기 값, round() : 반올림
  
> **cv2.equalizeHist()** 함수 제공 ==> ***Grayscale 영상만 입력 가능!!***

In [7]:
# 히스토그램 평활화 구현 ==> cv2.equalizeHist()

def histogram_equalization():
    src = cv2.imread('hawkes.bmp', cv2.IMREAD_GRAYSCALE)
    
    dst = cv2.equalizeHist(src)
    
    cv2.imshow('src', src)
    cv2.imshow('srcHist', getGrayHistImage(calcGrayHist(src)))
    
    cv2.imshow('dst', dst)
    cv2.imshow('dstHist', getGrayHistImage(calcGrayHist(dst)))
    cv2.imwrite('dstHist_equalized.png', getGrayHistImage(calcGrayHist(dst)))
    
    cv2.waitKey()
    cv2.destroyAllWindows()
    
histogram_equalization()

1. srcHist  
![src](srcHist_img.png)  
  
2. histogram_stretching   
![src](dstHist_img.png)  
  
3. histogram_equalization  
![src](dstHist_equalized.png)  